In [13]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [14]:
import os
import glob
import tqdm
import textgrids
import pandas as pd

from main import *

In [15]:
import tarfile
import shutil
import fsspec
from utils import generate_txt, get_subset_df, genorate_pps_df, make_tarfile

chunk = 10
generate_subset_tsv = True
pps_df_dir = '/home/knoriy/split_peoples_speech/pps_train.tsv'

root_path = '/home/knoriy/split_peoples_speech/'
dataset_name = 'pps_train'

metadata_dir = "/mnt/knoriy/metadata.json"
tar_dir = "/mnt/knoriy/pps_train.tar"

# init Dirs
dataset_root_path = os.path.join(root_path, f'{dataset_name}')
dataset_textgrid_path = os.path.join(root_path, f'{dataset_name}_textgrids')
dataset_split_path = os.path.join(root_path, f'{dataset_name}_split')
s3 = fsspec.filesystem('s3')
s3_dest = f's-laion/peoples_speech/{dataset_name}_tars/'

In [16]:
if os.path.isfile(pps_df_dir):
    pps_df = pd.read_csv(pps_df_dir, sep='\t', header=None, names=['audio_filepath', 'text'])
else:
    pps_df = genorate_pps_df(metadata_dir)
    pps_df.to_csv(pps_df_dir, sep='\t', header=None, index=False)

In [17]:
pps_df

,audio_filepath,text
0,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,is there anything to wear it on this one or ha...
1,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,i thought the law was that we don't have juris...
2,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,review it yes you do yes you do
3,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,this you know what happened there was a single...
4,gov_DOT_uscourts_DOT_ca9_DOT_04-56618/gov_DOT_...,motion to abandon the quantity because nothing...
...,...,...
4320997,The_Capture_movie/Capture_DOT_mp3_00011.flac,reward of all our marriage
4320998,The_Capture_movie/Capture_DOT_mp3_00012.flac,so i traveled back the way i'd come on it this...
4320999,The_Capture_movie/Capture_DOT_mp3_00013.flac,you state your business mister
4321000,The_Capture_movie/Capture_DOT_mp3_00014.flac,come out yet i don't want to be just that


In [18]:
src_file_obj = tarfile.open(tar_dir, mode='r')
print('opening file: This may take some time\n')

file_names_full_list = src_file_obj.getnames()
file_names_full_list = [i for i in file_names_full_list if '.flac' in i]

opening file: This may take some time



In [11]:
import time

for i in tqdm.tqdm(range(0, len(file_names_full_list), chunk), desc='Chunks remaining: '):
    for file_name in tqdm.tqdm(file_names_full_list[i:i + chunk], desc="Extracting Files: "):
        src_file_obj.extract(file_name, f'./{dataset_name}/')

    if generate_subset_tsv == True:
        df = get_subset_df(f'{dataset_root_path}/**/*.flac', pps_df)

    # Save transcript to file
    save_all_text_to_file(df, dataset_name)

    # Convert Flac to wav
    convert_all_to_wav(df, os.path.join(root_path, dataset_name))

    # Get audio text alignments and split audio
    generate_textgrids(os.path.join(root_path, dataset_name))
    split_all_audio_files(dataset_textgrid_path, dataset_root_path)

    # Upload Split files to s3
    tar_file_path = make_tarfile(f'{dataset_split_path}', f'{dataset_root_path}/{i}.tar')
    s3.put(tar_file_path, s3_dest)
    print('File Uploaded to: ', os.path.join(s3_dest, os.path.basename(tar_file_path)))
    time.sleep(100)
    if i == 20:
        break
    shutil.rmtree(dataset_root_path)
    shutil.rmtree(dataset_textgrid_path)
    shutil.rmtree(dataset_split_path)


Total Files found 4321002


Converting .flac files to .wav: 100%|██████████| 10/10 [00:01<00:00,  6.76it/s]
Traceback (most recent call last):
  File "/home/knoriy/miniconda3/envs/aligner/bin/mfa", line 7, in <module>
    from montreal_forced_aligner.command_line.mfa import main
  File "/home/knoriy/miniconda3/envs/aligner/lib/python3.10/site-packages/montreal_forced_aligner/__init__.py", line 4, in <module>
    import montreal_forced_aligner.acoustic_modeling as acoustic_modeling
  File "/home/knoriy/miniconda3/envs/aligner/lib/python3.10/site-packages/montreal_forced_aligner/acoustic_modeling/__init__.py", line 7, in <module>
    from montreal_forced_aligner.acoustic_modeling.base import AcousticModelTrainingMixin  # noqa
  File "/home/knoriy/miniconda3/envs/aligner/lib/python3.10/site-packages/montreal_forced_aligner/acoustic_modeling/base.py", line 20, in <module>
    from montreal_forced_aligner.alignment import AlignMixin
  File "/home/knoriy/miniconda3/envs/aligner/lib/python3.10/site-packages/montreal_for

In [12]:
shutil.rmtree('/home/knoriy/split_peoples_speech/pps_train/0.tar')

NotADirectoryError: [Errno 20] Not a directory: '/home/knoriy/split_peoples_speech/pps_train/0.tar'

In [ ]:
# src_file_obj.close()

# AWS S3

In [ ]:
%%bash

nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-07a8f0d3-6d27-4299-887a-dc12a6d72f8d-c000.tar?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A30%3A22Z/-1/host/444b9c082ceffd10f38bb965679ed9ec12202836831e111dd193fde281062d26 -O - | aws s3 cp - s3://s-laion/peoples_speech/train_clean_pps.tar" &;
nohup sh -c "wget https://the-peoples-speech-west-europe.bj.bcebos.com/part-00000-4e132642-c01c-4db6-9db0-a1e19193f6f8-c000.json?authorization=bce-auth-v1/0ef6765c1e494918bc0d4c3ca3e5c6d1/2021-12-03T06%3A31%3A22Z/-1/host/d7dacf3c31d2e3670d82727636ce234be27a9128df7a80883b84b4a3d8c7f6c0 -O - | aws s3 cp - s3://s-laion/peoples_speech/Manifest.json" &;
